In [1]:
import pandas as pd
from tqdm.auto import tqdm
import numpy as np

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', 300)

In [2]:
import json

with open("../data/fix_ipc_class_dict.json") as json_file:
    ipc_class_dict = json.load(json_file)

In [3]:
#df = pd.read_csv("../data/subest1_filtrato.csv")
df = pd.read_csv("../data/data_subset1.csv")

In [4]:
df.title = df.title.str.lower()
df.abstract = df.abstract.str.lower()
df.claims = df.claims.str.lower()
df.ipc_classes = df.ipc_classes.apply(lambda x: x.replace(",", ", ") if type(x)==str else "")

In [5]:
df.head()

,filename,filing_date,publication_date,priority_date,ipc_classes,assignee,inventors,docdb_family_id,title,abstract,claims
0,US2009216698_A1.txt,2008-02-22,2009-08-27,2008-02-22,"G06F15/18, G06N5/02","XEROX CORPORATIONANDREOLI, JEAN-MARCBOUCHARD, GUILLAUME","ANDREOLI, JEAN-MARCBOUCHARD, GUILLAUME",40999265,temporal events analysis employing tree induction,an events analysis method comprises: optimizing respective to a set of training data a set of branching transition likelihood parameters associating parent events of type k with child events of type k' in branching processes; inferring a most probable branching process for a set of input data co...,1. an events analyzer comprising: a probabilistic branching process learning engine configured to optimize respective to a set of training data a set of branching transition likelihood parameters associating parent events of type k with child events of type k′ in branching processes; and a proba...
1,US2020018760_A1.txt,2019-09-23,2020-01-16,2007-03-27,G01N33/574,IMMUNOVIA,"BORREBAECK, CARL, ARNE, KRISTERWINGREN, LARS, BERTIL, CHRISTER",39493874,protein signature/markers for the detection of adenocarcinoma,the present invention provides a method for determining the presence of pancreatic adenocarcinoma in an individual and/or for determining the survival time of an individual afflicted with pancreatic adenocarcinoma comprising the steps of: (a) providing a serum or plasma sample to be tested; and ...,1. a method for determining the presence of pancreatic adenocarcinoma in an individual comprising the steps of: a) providing a serum or plasma sample to be tested; b) determining a protein signature of the test sample by measuring the presence and/or amount in the test sample of one or more prot...
2,EP3340117_A1.txt,2017-11-15,2018-06-27,2016-12-20,"G06N3/04, G06N3/063",INTEL CORPORATION,"LIN, TSUNG-HAN",60327233,unsupervised learning using neuromorphic computing,"a spiking neural network (snn) is implemented on a neuromorphic computers and includes a plurality of neurons, a first set of the plurality of synapses defining feed-forward connections from a first subset of the neurons to a second subset of the neurons, a second subset of the plurality of syna...","1. a method comprising: defining a spiking neural network (snn) comprising a plurality of artificial neurons interconnected by a plurality of artificial synapses, wherein the snn is to comprise: a first subset of the plurality of synapses to define feedforward connections from a first subset of ..."
3,WO2019070570_A1.txt,2018-10-01,2019-04-11,2017-10-06,A61B5/103,TELLUS YOU CARE,"HSU KEVINCOKE, TANIA",65993331,non-contact activity sensing network for elderly care,"determining a physical state of a person includes detecting positions of different portions of the person, transforming detected positions of the person into a point cloud having a density that varies according to movement of each of the portions, correlating movement and position data from the ...","1. a method of determining a physical state of a person, comprising:detecting positions of different portions of the person;transforming detected positions of the person into a point cloud having a density that varies according to movement of each of the portions;correlating movement and positio..."
4,US2016259635_A1.txt,2015-03-04,2016-09-08,2015-03-04,"G06F15/18, G06F17/27, G06F8/65, G06F8/71, G06F9/44, G06F9/445, G06N99/00",IBM (INTERNATIONAL BUSINESS MACHINES CORPORATION),"EL MAGHRAOUI, KAOUTARJANN, JOEFONPATTNAIK, PRATAP C.PICKOVER CLIFFORD A.",56849635,software patch management incorporating sentiment analysis,"a method and system are provided. the method includes generating, by a machine-based sentiment prediction generator, respective machine-determined sentiment predictions for each of a plurality of software patches using sentiment analysis. the method further includes setting, by a sentiment-based...","1. 1-18. (canceled)19. a computer program product for softwar

In [6]:
len(df)

216986

<h3>Keyword della professoressa:</h3>

<img src="keyword prof.png" style="width:800px;"/>

In [40]:
keywords_other = []
"""["molecule",
                  "particle", 
                  "pharmacological",
                  "technolog",
                  "nano(-| )?tech(nolog)?",
                  "alter(ing)?",
                  "nano",
                  "nano(-| )?bot",
                  "nano(-| )?emulsion",
                  "nano(-| )?particle",
                  "nano(-| )?tube",
                  "nano(-| )?sphere",
                  "quantum",
                  "graphene",
                  "cloud",
                  "software",
                  "product"
                 ]"""

"""
Possiamo cercare i vincoli espressi dalla professoressa sia in "stile pandas" 
    (cercare documenti che contengono sia "body" che "care": df[(df.body >= 1) & (df.care >= 1)])
    
    che tramite l'uso delle espressioni regolari per implmentare l'operatore che l'Ing Ghironi 
    ha chiamto NEAR. Ad esempio per cercare "skin NEAR care" possiamo aggiungere alla lista sotto
    "lip(\w|\W){1,20}?stick". Agli estremi ci sono le 2 parole in cui siamo interessati, 
    "(\w|\W){1,20}?" serve invece a dire "permetti la presenza tra 
    0 e 20 caratteri o numeri tra le 2 parole". Se volete fare dei test: https://pythex.org/
"""

keywords_cosmetics = ["cosmetic",
                      "beauty", 
                      "body", 
                      "skin", 
                      "nail", 
                      "fingernail", 
                      "cleans(e|ing)", # 
                      "manicur(e|ing)", #manicure and manicuring
                      "pedicur(e|ing)"
                      "varnish", 
                      "polish",
                      "nail(s| |-)*polish",
                      "lip(s)?",
                      "lip( |-)?stic( |k)+",
                      "blush",
                      "mascara",
                      "make(s| |-)*up",#makes up, make-up
                      "cream",
                      "soap",
                      "toothpaste",
                      "sunscreen",
                      "lotion",
                      "deodorant",
                      "care",
                      "lip(s| |-)*(care|polish)",
                      "skin(s| |-)*(care|polish)",
                      "skin(s| |-)*(care|polish)",
                      "nail(s| |-)*(care|polish)",
                      "baby(s| |-)*(care|polish)",
                      "anti(-)?ag(e|ing)",
                      "colour|color",
                      "finger"
                     ]

### Controllo IPC
##### IPC A45D+

HAIRDRESSING OR SHAVING EQUIPMENT ; EQUIPMENT FOR COSMETICS OR COSMETIC TREATMENTS, e.g. FOR MANICURING OR PEDICURING

In [165]:
df_selected = pd.DataFrame()

In [166]:
df_candidate = df[df.ipc_classes.str.count("(^| )A45D")>0][["title", "ipc_classes","claims"]] 
df_candidate

,title,ipc_classes,claims
2674,"method and device for determining a degree of damage of hair, and method for determining a user-specific hair treatment agent","A45D44/00, A61B5/00, A61B5/103","1. a method for determining a degree of damage of hair, comprising the following steps: during the exposure of a hair sample to near-infrared and/or infrared light, recording a spectrum of at least part of the near-infrared and/or infrared light that has interacted with the hair sample; comparin..."
3240,system for dispensing at least one makeup product and method for dispensing and evaluating makeup,"A45D40/00, A45D40/26, A45D44/00, B01F13/10, B01F15/02, B01F5/06, G01J3/46","1. a system for dispensing at least one makeup product and that can be manipulated in one hand, the system comprising: a cup; and a dispenser for filling the cup from a bottom of the cup with at least two base products, wherein: the cup is secured to the dispenser at least while the cup is being..."
7691,systems and methods for measuring and categorizing colors and spectra of surfaces,"A45D44/00, A61B5/00, G01J3/02, G01J3/46",1. a color categorization method comprising the steps of:taking at least one color or spectral measurement of a color of a surface using a color or spectral measurement device;mapping the at least one color or spectral measurement into a multi-dimensional feature space; andclassifying the mapped...
8026,grooming instrument configured to monitor hair loss/growth,"A45D24/16, A46B15/00, A61B5/00, A61B5/053, G06N3/08","1. an instrumented comb having a handle, the instrumented comb comprising: a plurality of bristles protruding from the handle; a power source; and a controller enclosed in the handle receiving signals from: a contact sensor to detect contact with a scalp, and a magnetic compass to determine loca..."
8521,use of artificial intelligence in providing beauty advice,"A45D44/00, A61B5/103, G06F15/00, G06F15/18, G06F17/30, G06F19/00, G06F7/00, G06Q30/06, G06T11/00","1. a method of providing beauty advice, the method comprising: receiving user-specific information; accessing a data structure containing information reflecting relationships between categories of user-specific information and beauty advice; comparing, using an artificial intelligence engine, th..."
...,...,...,...
210786,smart mirror system and methods of use thereof,"A45D44/00, A47G1/02, G06F3/01, G06T7/70","what is claimed is:1. a system comprising:a frame;a mirror coupled to the frame;an electronic display device coupled to the frame such that the electronic display device is positioned adjacent to at least a portion of the mirror; a camera coupled to the frame, the camera having a field of view; ..."
213268,historical beauty record,"A45D44/00, A61B5/103, G06F19/00, G06T11/00","1. a beauty care method, comprising: obtaining first information representative of an external body condition of a subject; obtaining second information representative of the external body condition of the subject after the subject uses a beauty product; facilitating storage of information refle..."
214209,image processing device and image processing method,"A45D44/00, A61F13/02, B41M3/00, B41M3/12, G06T11/00, G06T7/00, G06T7/11, G06T7/90","1. an image processing apparatus that generates image data for use in printing an image on a sheet stickable to a skin, comprising: an image acquirer that acquires a first skin image obtained by capturing an image of a skin at a first timing and a second skin image obtained by capturing an image..."
215818,methods and systems for predicting and/or tracking changes in external body conditions,"A45D44/00, A61B5/00, A61B5/103, G06F19/00, G06Q50/00, G06T1/00, G06T11/00","1. a method for predicting evolution of at least one condition of an external body portion of a subject, the method comprising: receiving first data, wherein the first data is representative of at least one condition of an external body portion of a subject in a first time frame; receiving secon..."


##### IPC C11B9/00 (riportato nelle slide come C11B/009)

In [57]:
df_candidate = df[df.ipc_classes.str.count("(^| )C11B9/00")>0][["title", "ipc_classes","claims"]] 
df_candidate

,title,ipc_classes,claims
5337,method of making perfumed goods,"C11B9/00, C11D3/50","1. a method comprising: a.) using an odt value determined by mathematical modelling to select one or more perfume raw materials; b.) combining and/or processing said one or more perfume raw materials with one or more additional materials to form a product.2. the method of claim 1, wherein said p..."
8543,olfactory signature and odorant mixture having the same,"C11B9/00, C11B9/02, G01N33/00, G06F17/30, G06F19/00","1. an odorant mixture, comprising n odorant components wherein n equals at least 20, each odorant component being characterized by a multidimensional vector of attributes, such that the mixture is characterized by n respective multidimensional vectors, wherein a z score of an average of characte..."
42547,olfactory signature and odorant mixture having the same,"C11B9/00, C11B9/02, G01N33/00, G06F17/30, G06F19/00","1. an odorant mixture, comprising n odorant components wherein n equals at least 20, each odorant component being characterized by a multidimensional vector of attributes, such that the mixture is characterized by n respective multidimensional vectors, wherein a z score of an average of characte..."
48466,olfactory signature and odorant mixture having the same,"C11B9/00, C11B9/02, G01N33/00, G06F17/30, G06F19/00","1. a method of determining an ability of an odorant mixture having n odorant components to mask other odors, the method being executable by a data processor and comprising: accessing a database having a group of m odorant components; for each odorant component, obtaining a multidimensional vecto..."
106943,method of making perfumed goods,"A61K8/33, A61K8/34, A61K8/35, A61K8/36, A61K8/368, A61Q15/00, A61Q19/10, C11B9/00, G06F17/10, G06F19/00, G06Q99/00","1. a method comprising: a.) using a malodour reduction value determined by mathematical modelling to select one or more perfume raw materials; b.) combining and/or processing said one or more perfume raw materials with one or more additional materials to form a product.2. the method of claim 1, ..."
135822,method of resolving allergens in perfume ingredients,"B01J20/26, B01J20/281, C11B9/00, G01N30/46, G01N30/86, G16C20/20","1. a method of resolving defined known or suspected allergens in a complex mixture of perfume ingredients using two-dimensional gas chromatography (2d-gc), wherein the combination of first and second stationary phases is selected on the basis of a clustering analysis of defined known or suspecte..."
174990,method of making perfumed goods,"A61L9/01, A61Q19/00, C11B9/00, C11D3/22, C11D3/50",what is claimed is:1. a method comprising:a.) using a malodour reduction value determined by mathematical modelling toselect one or more perfume raw materials;b.) combining and/or processing said one or more perfume raw materials with one or more additional materials to form a product. 2. the me...
175028,method of making perfumed goods,"A61K8/33, A61K8/34, A61K8/35, A61K8/37, A61Q13/00, A61Q15/00, A61Q19/10, C11B9/00, C11D11/00, C11D3/50, G06F17/50","1. a method comprising: a.) using a malodour reduction value determined by mathematical modelling to select one or more perfume raw materials; b.) combining and/or processing said one or more perfume raw materials with one or more additional materials to form a product.2. the method of claim 1, ..."
210149,"methods for identifying, compounds identified and compositions thereof","A23L27/00, C11B9/00","1. a flavor/fragrance composition, comprising an effective amount of one or more compounds selected from tables 1-42.2. the flavor/fragrance composition of claim 1, wherein the composition is formulated as a spray, lotion, foam, gel, suspension, or emulsion.3. the flavor/fragrance composition of..."


## Analisi Per keyword
#### Vedi slides Ing. Ghironi

In [59]:
for key in tqdm(keywords_cosmetics):
    df[key] = df.title.str.count(key).fillna(0) + df.abstract.str.count(key).fillna(0) + df.claims.str.count(key).fillna(0)
    
for key in tqdm(keywords_other):
    df[key] = df.title.str.count(key).fillna(0) #+ df.abstract.str.count(key).fillna(0) + df.claims.str.count(key).fillna(0)

  0%|          | 0/31 [00:00<?, ?it/s]

0it [00:00, ?it/s]

In [141]:
df["total_cosmetics"] = 0
for key in keywords_cosmetics:
    if key == "body" or key == "lip(s)?": continue
    df["total_cosmetics"] += df[key]
    
df["total_other"] = 0
for key in keywords_other:
    df["total_other"] += df[key]

In [143]:
keywords_cosmetics

['cosmetic',
 'beauty',
 'body',
 'skin',
 'nail',
 'fingernail',
 'cleans(e|ing)',
 'manicur(e|ing)',
 'pedicur(e|ing)varnish',
 'polish',
 'nail(s| |-)*polish',
 'lip(s)?',
 'lip( |-)?stic( |k)+',
 'blush',
 'mascara',
 'make(s| |-)*up',
 'cream',
 'soap',
 'toothpaste',
 'sunscreen',
 'lotion',
 'deodorant',
 'care',
 'lip(s| |-)*(care|polish)',
 'skin(s| |-)*(care|polish)',
 'skin(s| |-)*(care|polish)',
 'nail(s| |-)*(care|polish)',
 'baby(s| |-)*(care|polish)',
 'anti(-)?ag(e|ing)',
 'colour|color',
 'finger']

##### cosmetic

In [144]:
keyword = keywords_cosmetics[0]
keyword

'cosmetic'

In [145]:
df[df[keyword] > 0][["title", "ipc_classes","abstract", "claims"]]

,title,ipc_classes,abstract,claims
1827,in vitro toxicogenomics for toxicity prediction using probabilistic component modeling and a compound-induced transcriptional response pattern,"C12Q1/68, C12Q1/6876, G06F19/12, G06F19/20, G16B20/00, G16B25/00, G16B5/00, G16C20/10, G16C20/30, G16C20/70","a novel method to predict toxicity and dose-dependent effects of an agent based on transcriptomic data analysis, by determining a predictive toxicogenomics space (ptgs) score. the ptgs score helps to predict and model the toxicity of compounds typically consisting of chemicals, pharmaceuticals, ...","1. a method of defining a predictive toxicogenomics space, ptgs, score indicative of a toxicity prediction for an agent, said method comprising: transforming gene data defining, for each instance of a plurality of instances, differential gene expression of genes in a test biological sample induc..."
3240,system for dispensing at least one makeup product and method for dispensing and evaluating makeup,"A45D40/00, A45D40/26, A45D44/00, B01F13/10, B01F15/02, B01F5/06, G01J3/46","a system for dispensing at least one makeup product, including a cup and a dispenser for filling the cup from the bottom with at least one product, the cup being secured to the dispenser at least while it is being filled with said product. the method for dispensing and evaluating makeup, includi...","1. a system for dispensing at least one makeup product and that can be manipulated in one hand, the system comprising: a cup; and a dispenser for filling the cup from a bottom of the cup with at least two base products, wherein: the cup is secured to the dispenser at least while the cup is being..."
5009,automated characterization and classification of microoganisms,"G01N21/65, G01N33/569","the present invention relates to an instrument, method, use and software program to obtain information rapidly about microorganisms that may spread uncontrolled in hospitals, water supply, food or when used in bio terrorism. vibrational spectroscopy provides data to a computer linked to one or m...","1. an instrument comprising a vibrational spectrometer, a processing means, a first spectral database [10] and a first information database [200], for obtaining information about a microorganism in a sample of interest by establishing vibrational spectroscopic similarity of the microorganism in ..."
6806,methods and systems for dynamic content provisioning,"G06F16/435, G06F16/438, G06F16/483, G06Q50/00","a method, system, and computer program product for dynamic content provisioning, such as for online marketing and product promotion, may include collecting user content from sources based upon collection parameters, and organizing the collected user content for optimization. there may also be th...","1. a system for dynamic content provisioning, comprising: at least one non-transitory memory configured to store computer program code instructions; and at least one processor configured to execute the computer program code instructions to: collect user content from one or more sources based on ..."
7363,personal appliance,"B26B19/38, B26B21/40",a personal appliance. the personal appliance includes a handle having an implement connecting structure. an implement is connected to the implement connecting structure. an implement displacement sensor is positioned in the handle. the implement displacement sensor measures a displacement of the...,"what is claimed is:1. a personal appliance comprising:a. a handle comprising an implement connecting structure;b. an implement connected to the implement connecting structure;c. an implement displacement sensor positioned in the handle, the implement displacement sensor measuring a displacement ..."
...,...,...,...,...
214657,"methods and systems for microbiome characterization, monitoring and treatment","C12Q1/68, C12Q1/689","the present disclosure provides methods for profiling a microbiome and therapeutic compositions for treatment. additionally, the method

In [146]:
#escludiamo tutti i sistemi per "raccomandare un prodotto"

df_candidate = df[(df[keyword] > 0) 
                  & (df.ipc_classes.str.count("G06")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("G01")==0) #Escludo "MEASURING; TESTING"
                  & (df.ipc_classes.str.count("A61")==0) #MEDICAL OR VETERINARY SCIENCE; HYGIENE
                  & (df.ipc_classes.str.count("(^| )A")>0)]

df_candidate[["filename", "title", "ipc_classes","claims"]] 

,filename,title,ipc_classes,claims
21464,US2020054018_A1.txt,methods for assessing repellant quality of organic materials and methods and compositions for repelling arthropods,"A01N31/02, A01N31/06, A01N33/10, A01N35/02, A01N35/04, A01N35/06, A01N35/10, A01N37/10, A01N37/18, A01N37/20, A01N37/22, A01N37/36, A01N37/44, A01N37/46, A01N43/12, A01N43/16, A01N43/32, A01N43/38, A01N43/40, A01N43/42, A01N43/50, A01N43/54, A01N43/60, A01N43/78, A01N43/90, A01N47/16, A01N47/42,...","1. a method of repelling an arthropod, comprising exposing the arthropod with a repelling composition, wherein the repelling composition comprises one or more compounds selected from the group consisting of methyl n,n-dimethyl anthranilate, ethyl anthranilate, butyl anthranilate, and 2,3-dimethy..."
27784,EP3733278_A1.txt,"moisture control device, moisture control method, program, storage medium, generated substance, product, device and equipment","A23L3/26, A23L3/30, A23L3/32, A23L5/30, A47J37/12, B01J19/08, B01J19/10, B01J19/12","1. a moisture control apparatus, wherein a predetermined voltage or current including a dc component and/or an ac component is applied to at least one electrode that generates at least one of an electric field, a magnetic field, an electromagnetic field, electromagnetic waves, sound waves, and u..."
73457,US2017079274_A1.txt,methods for assessing repellant quality of organic materials and methods and compositions for repelling arthropods,"A01N31/02, A01N31/06, A01N33/10, A01N35/02, A01N35/04, A01N35/06, A01N35/10, A01N37/10, A01N37/18, A01N37/20, A01N37/22, A01N37/36, A01N37/44, A01N37/46, A01N43/12, A01N43/16, A01N43/32, A01N43/38, A01N43/40, A01N43/42, A01N43/50, A01N43/54, A01N43/60, A01N43/78, A01N43/90, A01N47/16, A01N47/42,...","1. 1-21. (canceled)22. a method of repelling an arthropod, comprising exposing the arthropod to a composition comprising one or more compounds selected from the group consisting of:23. the method of claim 22, wherein the composition is applied on the body of one or more vertebrates or one or mor..."
90992,US10292396_B2.txt,methods for assessing repellant quality of organic materials and methods and compositions for repelling arthropods,"A01N31/02, A01N31/06, A01N33/10, A01N35/02, A01N35/04, A01N35/06, A01N35/10, A01N37/10, A01N37/18, A01N37/20, A01N37/22, A01N37/36, A01N37/44, A01N37/46, A01N43/12, A01N43/16, A01N43/32, A01N43/38, A01N43/40, A01N43/42, A01N43/50, A01N43/54, A01N43/60, A01N43/78, A01N43/90, A01N47/16, A01N47/42,...","1. a method of repelling an arthropod, comprising exposing the arthropod to a composition comprising one or more compounds selected from the group consisting of:2. the method of claim 1, wherein the composition is applied on the body of one or more vertebrates or one or more plants to expose the..."
95067,US9491942_B2.txt,methods for assessing repellant quality of organic materials and methods and compositions for repelling arthropods,"A01N31/02, A01N33/10, A01N35/06, A01N35/10, A01N37/18, A01N37/20, A01N37/22, A01N37/36, A01N37/44, A01N37/46, A01N43/32, A01N43/38, A01N43/40, A01N43/42, A01N43/50, A01N43/54, A01N43/60, A01N43/78, A01N43/90, A01N47/16, A01N47/42, A01N47/46, C08G16/02","1. a method of repelling an arthropod, comprising exposing the arthropod to a repelling composition, wherein the repelling composition comprises at least one compound of formula i: wherein: r1 is selected from the group consisting of c6-c12 alkyl, c4-c12 alkenyl, aryl, and arylalkyl; r2 and r3 a..."
98335,WO2015013214_A2.txt,"methods and systems for microbiome characterization, monitoring and treatment","A01N63/00, C12N1/20, C12Q1/68, C40B40/12","what is claimed is:1. a method of classifying a microbe, comprising:obtaining a nucleic acid sequence of a 16s or 23s ribosomal subunit contained within a single read length of a first microbe; andcomparing said nucleic acid sequence of a first microbe to a reference; andidentifying the first mi..."
106389,US2012003728_A1.txt,scalable portable sens

In [147]:
df_selected = pd.concat([df_selected, df_candidate])
len(df_selected)

13

##### beauty

In [148]:
keyword = keywords_cosmetics[1]
keyword

'beauty'

In [149]:
df_candidate = df[(df[keyword] > 0)
                  & (df.ipc_classes.str.count("G06")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("G01")==0) #Escludo "MEASURING; TESTING"
                  & (df.ipc_classes.str.count("A61")==0) #MEDICAL OR VETERINARY SCIENCE; HYGIENE
                 ].query("title not in @df_selected")

df_candidate[["title", "ipc_classes","abstract", "claims"]]

,title,ipc_classes,abstract,claims
5337,method of making perfumed goods,"C11B9/00, C11D3/50",the present invention relates to methods of designing and making perfumed products and perfume raw materials for use in products and perfume raw materials selected by such methods and the use of same.,"1. a method comprising: a.) using an odt value determined by mathematical modelling to select one or more perfume raw materials; b.) combining and/or processing said one or more perfume raw materials with one or more additional materials to form a product.2. the method of claim 1, wherein said p..."
7363,personal appliance,"B26B19/38, B26B21/40",a personal appliance. the personal appliance includes a handle having an implement connecting structure. an implement is connected to the implement connecting structure. an implement displacement sensor is positioned in the handle. the implement displacement sensor measures a displacement of the...,"what is claimed is:1. a personal appliance comprising:a. a handle comprising an implement connecting structure;b. an implement connected to the implement connecting structure;c. an implement displacement sensor positioned in the handle, the implement displacement sensor measuring a displacement ..."
27784,"moisture control device, moisture control method, program, storage medium, generated substance, product, device and equipment","A23L3/26, A23L3/30, A23L3/32, A23L5/30, A47J37/12, B01J19/08, B01J19/10, B01J19/12","provided are a moisture control apparatus, a moisture control method, a program, a storage medium, a produced object, a product, an apparatus, and a facility that can improve the characteristics of an object through moisture control. in a moisture control apparatus according to one aspect of the...","1. a moisture control apparatus, wherein a predetermined voltage or current including a dc component and/or an ac component is applied to at least one electrode that generates at least one of an electric field, a magnetic field, an electromagnetic field, electromagnetic waves, sound waves, and u..."
31462,training system and device,"A45D1/00, G09B19/00, G09B19/24, G09B5/04",a beauty product device is disclosed which includes a variety of sensors for sensing the way in which a user is operating the device. the sensor data can be processed and compared with pre-stored data representative of a desired style and feedback messages generated and provided back to the user...,"1. a beauty product system for performing a treatment on a user, the system comprising:an input for allowing the user to specify a desired treatment to be performed; a beauty product device that is manipulate by the user to perform the desired treatment on the user; the beauty product device hav..."
62406,personal appliance,"B26B19/38, B26B21/40, B26B21/52",a personal appliance. the personal appliance includes a handle having an implement connecting structure. an implement is connected to the implement connecting structure. an implement displacement sensor is associated with the handle. the implement displacement sensor measures a displacement of t...,"1. a personal appliance comprising: a. a handle comprising an implement connecting structure; b. an implement connected to the implement connecting structure; c. an implement displacement sensor associated with the handle, the implement displacement sensor measuring a displacement of the impleme..."
140736,personal appliance,"B26B19/38, B26B21/40",a personal appliance. the personal appliance includes a handle having an implement connecting structure. an implement is connected to the implement connecting structure. an implement displacement sensor is positioned in the handle. the implement displacement sensor measures a displacement of the...,"1. a personal appliance comprising: a. a handle comprising an implement connecting structure; b. an implement connected to the implement connecting structure; c. an implement displacement sensor positioned in the handle; and d. a follower posi

In [150]:
df_selected = pd.concat([df_selected, df_candidate])
len(df_selected)

22

##### body (troppo rumore)

In [151]:
keyword = keywords_cosmetics[2]
keyword

'body'

In [152]:
df_candidate = df[(df[keyword] > 0)
                  & (df.ipc_classes.str.count("G06")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("G01")==0) #Escludo "MEASURING; TESTING"
                  & (df.ipc_classes.str.count("A61")==0) #MEDICAL OR VETERINARY SCIENCE; HYGIENE
                  & (df.ipc_classes.str.count("A63H")==0) #TOYS, e.g. TOPS, DOLLS, HOOPS OR BUILDING BLOCKS
                  & (df.ipc_classes.str.count("G09")==0) #EDUCATION; CRYPTOGRAPHY; DISPLAY; ADVERTISING; SEALS
                  & (df.ipc_classes.str.count("G10")==0) #MUSICAL INSTRUMENTS; ACOUSTICS
                  & (df.ipc_classes.str.count("(^| )A")>0)
                  ].query("title not in @df_selected")[["title", "ipc_classes","abstract", "claims"]]

df_candidate

,title,ipc_classes,abstract,claims
566,systems and methods for data-driven movement skill training,"A63B24/00, A63B69/40, A63B71/06",a data-driven movement skill training system is disclosed. the system uses movement skill assessment and diagnostics at distinct levels of the human movement system hierarchy to specify training goals for a user. the system may provide various augmentations that are synthesized to help the user ...,"1. an apparatus for movement skill training, the apparatus comprising: a sensor system comprising one or more sensors configured to obtain movement data for a subject performing an activity; a processor system in communication with the one or more sensors, the processor system having a microproc..."
2654,mobile robot and control method therefor,"A47L9/28, B25J11/00, B25J19/02, B25J9/00, B25J9/16, G05D1/02",a mobile robot in one general aspect of the present invention includes: a travel drive unit configured to move a main body; an image acquisition unit configured to acquire an image of surroundings of the main body; a sensor unit having one or more sensors configured to sense an object during the...,1. a mobile robot comprising: a travel drive unit configured to move a main body; an image acquisition unit configured to acquire an image of surroundings of the main body; a sensor unit having one or more sensors configured to sense an object during the movement of the main body; a storage conf...
3478,connected food preparation system and method of use,"A23L5/10, A47J36/32, A47J37/06, F24C15/00, F24C3/12, F24C7/08, H04W4/80, H05B1/02","a connected oven, including a set of in-cavity sensors and a processor configured to automatically identify foodstuff within the cooking cavity, based on the sensor measurements; and automatically operate the heating element based on the foodstuff identity.","we claim:1. a connected oven, comprising: an oven body defining a cooking cavity, the oven body comprising an oven base opposing an oven top, the cooking cavity comprising a set of low reflectance surfaces; a set of optical sensors cooperatively defining a field of view, the set of optical senso..."
4902,dribbling and ball control sports training system and method,A63B69/00,"a sports training method is described comprising the following steps: classifying the dribbling and ball or puck control motions into modular predetermined motion sequences according to the sensomotor and technical skills developed, evaluating athlete's skill level in relation to dribbling and b...","1. a sports training method which comprises the following steps: classifying different dribbling and ball or puck control motions into predetermined motion sequences according to the sensomotor and technical skills developed, evaluating athlete's skill level in relation to dribbling and ball ..."
5260,a connected oven,"A23L5/10, A47J36/32, A47J37/06, F24C15/00, F24C3/12, F24C7/06, F24C7/08, H04W4/80, H05B1/02, H05B6/68","a connected oven, including a set of in-cavity sensors and a processor configured to automatically identify foodstuff within the cooking cavity, based on the sensor measurements; and automatically operate the heating element based on the foodstuff identity.","1. a connected oven (100), comprising: • an oven body defining a cooking cavity (200), the oven body comprising an oven base (250) and an oven top (230); • an oven door (210) mounted to the oven body, the oven door (210) comprising: • a transparent window coextensive with the cooking cavity (200..."
...,...,...,...,...
209972,mobile robot and control method therefor,"A47L9/28, B25J11/00, B25J19/02, B25J9/00, B25J9/16, G05D1/02",a mobile robot in one general aspect of the present invention includes: a travel drive unit configured to move a main body; an image acquisition unit configured to acquire a plurality of images by continuously photographing surroundings of the main body; a storage configured to store the plurali...,1. a mobile robot comprising: a travel drive unit configu

##### skin (da provare combinazioni)

In [153]:
keyword = keywords_cosmetics[3]
keyword

'skin'

In [154]:
df_candidate = df[(df[keyword] > 0)
                  & (df.ipc_classes.str.count("G06")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("G01")==0) #Escludo "MEASURING; TESTING"
                  & (df.ipc_classes.str.count("A61")==0) #MEDICAL OR VETERINARY SCIENCE; HYGIENE
                  & (df.ipc_classes.str.count("G09")==0) #EDUCATION; CRYPTOGRAPHY; DISPLAY; ADVERTISING; SEALS
                  & (df.ipc_classes.str.count("(^| )A")>0)
                 ].query("title not in @df_selected")

df_candidate[["title", "ipc_classes","abstract", "claims"]]

,title,ipc_classes,abstract,claims
538,ligands for odor receptors and olfactory neurons,"A01M1/10, A01N33/00, A23L27/10",the disclosure provides compounds useful as insect repellents and compositions comprising such repellents. the disclosure further provides insect traps and method for identifying ligands and cognates for biological molecules.,"1-41. (canceled)42. a composition, comprising one or more compounds selected from table 9, table 10, and any combination thereof, in a trap, a cream, a lotion, a spray, a dust, a vapor emitter, a candle, an oil, a wicked apparatus, a fan, a vaporizer, a perfume, a cologne, a fragrance, a deodora..."
2533,rowing exercise machines having a configurable rowing feel,"A63B21/00, A63B21/005, A63B21/22, A63B22/00, A63B24/00, A63B71/06","among other things, a rowing exercise machine includes a movable inertial element, an eddy current brake coupled to the movable inertial element, a rowing grip coupled to the movable inertial element, and control circuitry coupled to the eddy current brake to cause a resistance to motion of the ...","1. a rowing exercise machine comprising a movable inertial element, an eddy current brake coupled to the movable inertial element, a rowing grip coupled to the movable inertial element, and control circuitry coupled to the eddy current brake to cause a resistance to motion of the rowing grip dur..."
4603,creating music by concatenative synthesis,"A63H5/00, G04B13/00, G10H7/00, H03G5/00","automated creation of new music by listening is disclosed. a method to create new music may comprise listening to a plurality of music, learning from the plurality of music, and performing concatenative synthesis based on the listening and the learning to create the new music. the method may be ...",it is claimed:1. a method to create new music comprising: listening to a plurality of music learning from the plurality of music performing concatenative synthesis based on the listening and the learning to create the new music.2. the method of claim 1 wherein the music comprises at least one of...
11876,personally powered appliance,"A43B3/00, G08B21/02, H01L41/113, H04B1/38, H04Q9/00","systems and methods are disclosed for powering a processor and a wireless transceiver in a sole having a sole plate and one or more spring elements disposed in a forefoot area of the sole plate, each of said spring elements including a piezoelectric generator coupled thereto, wherein each of the...","1. an apparatus, comprising: a processor; a wireless transceiver coupled to the processor; a power supply coupled to the processor and wireless transceiver located proximal to a sole, said sole including: a sole plate; and one or more leaf spring elements extending at an angle from the sole plat..."
12941,devices for prevention of pressure ulcers,"A47C27/00, A47C27/08","a support surface, such as an underpad, for preventing pressure sores in a patient is a multi-layer textile assembly having a skin contact layer formed from a fabric of synthetic filament yarns woven to provide air permeability and moisture vapor permeability and having a thickness of less than ...","1. a support surface for preventing pressure ulcers in a patient, the support surface comprising a multi-layer composite fabric comprising: a skin contact layer comprising a woven fabric of synthetic filament yarns woven to provide air permeability and moisture vapor permeability and having a th..."
15513,computer-implemented question and answer game,A63F9/24,"a computer-implemented question and answer game based on using opinion polls that have already been carried out on a representative sample of people. results of the opinion polls and predefined questions are stored in a knowledge base associated with the system, which is typically implemented on...","1. a method of allocating a prize to at least one individual using a computer system having at least one host computer terminal for presenting and collecting -data, a question engine for sendi

In [155]:
df_selected = pd.concat([df_selected, df[df.title == "personal care device with camera"]])
len(df_selected)

24

##### skin (da provare combinazioni)

In [156]:
keyword = keywords_cosmetics[4]
keyword

'nail'

In [157]:
df_candidate = df[(df[keyword] > 0)
                  & (df.ipc_classes.str.count("G06")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  & (df.ipc_classes.str.count("G01")==0) #Escludo "MEASURING; TESTING"
                  & (df.ipc_classes.str.count("A61")==0) #MEDICAL OR VETERINARY SCIENCE; HYGIENE
                  & (df.ipc_classes.str.count("G16")==0)#NFORMATION AND COMMUNICATION TECHNOLOGY [ICT] SPECIALLY ADAPTED FOR SPECIFIC APPLICATION FIELDS
                  & (df.ipc_classes.str.count("G08")==0)#SIGNALLING
                  & (df.ipc_classes.str.count("G09")==0)#EDUCATION; CRYPTOGRAPHY; DISPLAY; ADVERTISING; SEALS
                  & (df.ipc_classes.str.count("H04")==0)#LECTRIC COMMUNICATION TECHNIQUE
                 ].query("title not in @df_selected")[["title", "ipc_classes","abstract", "claims"]]

df_candidate

,title,ipc_classes,abstract,claims
10744,"improved microlayer membranes, mproved battery separators, and related methods","B29C48/00, B29C48/08, B29C48/21, B32B27/08, B32B27/32, H01M2/14, H01M2/16","described herein is a multilayer microporous film or membrane that may exhibit improved properties, including improved dielectric break down and strength, compared to prior monolayer or tri-layer microporous membranes of the same thickness. the preferred multilayer microporous membrane comprises...","1. a battery separator for a lithium battery comprising: at least one microporous separator membrane or sub-membrane comprising a plurality of porous or microporous polymer microlayers or nanolayers, wherein at least one of the individual microlayers or nanolayers comprises a different or distin..."
14944,power center assembly having electrical connection-protection and optional detachable surface mount,"H01R13/516, H01R13/66, H01R13/70, H01R25/00, H02G3/18","a power center assembly having a hooded member for protecting an electrical connection between a peripheral device plug and its respective outlet and a mount member for detachably mounting the power center to a surface. in particular, a power center assembly is provided having a member for prote...","1. a power center assembly, comprising: a power strip, said power strip having a housing member and at least one outlet for powering at least one peripheral device, said housing member having a bottom surface; and mechanical means for preventing inadvertent mechanical and electrical disconnec..."
20938,automatic nail polish application system and method,"A45D29/00, A45D29/11, A45D29/12, A45D29/14, A45D34/04",the present disclosure relates to a robotic apparatus and methods for automatic nail polish application on natural or artificial finger or toe nails. the robot uses artificial intelligence (al) to identify and paints the nails. the robot uses depth sensors and computer vision to plan the movemen...,"1. a system for automatically polishing a nail of a user including:an end-effector havinga cartridge receiving unit,a cartridge including nail polish, anda nozzle;a sensor, to generate target signals from a target location;a nail determination unit, receiving said target signals, to automaticall..."
22515,system and methods for patient data and treatment management,,"described are systems and methods for the analysis of information, particularly information obtained from the mass spectroscopic (proteomic) analysis of biological samples and associated patient and clinical information. the systems and methods of the invention may be used for a variety of purpo...","what is-1cealmeo fs:11. a system, comprising: a system input to receive a quantity of proteomic data obtained from mass spectrometric analysis of a biological sample; a metadata repository to store a quantity of metadata that includes the quantity of proteomic data; a rules database to store at ..."
27088,interchangeable footwear system and method,"A43B21/36, A43B3/24","a footwear system (s) and corresponding methods (m1) and (m2) for providing a plethora of footwear fashion options via a plurality of interchangeable components, the interchangeability of which being facilitated by structurally stable quick-coupling and quick-decoupling techniques. the system (s...","1. an interchangeable footwear system, comprising: a plurality of interchangeable uppers; a plurality of interchangeable heels; and a quick-release device comprising a rigid slide reversibly fastenable to the plurality of interchangeable uppers and reversibly fastenable to the plurality of inter..."
40165,"improved microlayer membranes, improved battery separators, and related methods","B29C47/06, B32B27/00, H01M10/052, H01M2/14, H01M2/16, H01M2/18","described herein is a multilayer microporous film or membrane that may exhibit improved properties, including improved dielectric break down and strength, compared to prior monolayer or tri-layer microporous membranes 

In [158]:
df_selected = pd.concat([df_selected, df[df.title == "automatic nail polish application system and method"]]) 
len(df_selected)

25

##### fingernail

In [159]:
keyword = keywords_cosmetics[5]
keyword

'fingernail'

In [164]:
df_candidate = df[(df[keyword] > 0)
                 ].query("title not in @df_selected")[["title", "ipc_classes","abstract", "claims"]]


df_candidate #Contiene solo metodi relativi al miglioramento dei dispositivi touch

,title,ipc_classes,abstract,claims
587,platform for integrating back-end data analysis tools using schema,"G06F16/21, G06F3/0482, G06F8/34","in one embodiment of systems and methods described herein, a computing device is configured to generate and display a user interface including one or more selectable generic options; receive a selection of a data analysis function to perform machine learning; translate the selected function into...","1. a computer-implemented method, comprising: generating and displaying a user interface including one or more selectable generic options; receiving a selection of a data analysis function; translating the selected function into pre-configured commands for each back-end tool that are specific to..."
1432,system and method providing real-time assistance to drilling operation,"E21B45/00, E21B47/00, E21B47/06, E21B47/10, G06N3/04","a system for providing real-time assistance to a drilling operation drilling a bore in the earth, comprising: a computer; a first non-transitory computer-readable medium storing a first program that, when executed by the computer, causes the computer to: receive real-time raw data from sensors m...","1. a system for providing real-time assistance to a drilling operation drilling a bore in the earth, comprising: a computer; a first non-transitory computer-readable medium storing a first program that, when executed by the computer, causes the computer to: receive real-time raw data from sensor..."
2103,method and system for cleansing and de-duplicating data,"G06F12/02, G06F16/215, G06F16/2455, G06F17/30, G06N20/00, G06N99/00","method and system for cleansing and de-duplicating data in database are provided. the method includes filtering garbage records from a plurality of records based on data fields, and applying cleansing rules to create a cleansed database. a similarity vector is generated, where each vector corres...","1. a computer-implemented method for cleansing and de-duplicating data in a database, the computer-implemented method comprising: filtering garbage records from a plurality of records in the database based on data fields; creating a cleansed database by applying cleansing rules to remove the gar..."
4009,system and method for automatically suggesting rules for data stored in a table,"G06N5/02, G06N7/02, G06N7/04, G06N7/06, G06N7/08","a method and system are presented of automatically suggesting rules for data stored in a table, with the table comprising a plurality of columns. the table is profiled to identify a content type for each of one or more of the plurality of columns. a rule knowledge base is accessed to locate rule...","1. a method of automatically suggesting rules for data stored in a table, the table comprising a plurality of columns, the method comprising: for each of one or more of the columns that do not have a content type explicitly provided in the table, identifying, using at least one hardware processo..."
4133,system and method for forecasting industrial machine failures,"G05B11/32, G05B23/00, G06N20/00","a system and method for forecasting failures in industrial machines, including: receiving raw sensory inputs collected from at least one machine; generating a plurality of data features based on the raw sensory inputs; selecting from the plurality of data features a plurality of indicative data ...","what is claimed is:1. a method for forecasting failures in industrial machines, comprising:receiving raw sensory inputs collected from at least one machine;generating a plurality of data features based on the raw sensory inputs;selecting from the plurality of data features a plurality of indicat..."
...,...,...,...,...
212355,systems and methods venue visitation forecasting,"G06N20/00, G06N5/02, G06Q10/02","a system and method for the computerized forecasting of venue, visitation based on data comprising historic visitation data and a contextual data set comprising a set of defined factors, the system and method including the building

##### cleans(e|ing)

In [161]:
keyword = keywords_cosmetics[6]
keyword

'cleans(e|ing)'

In [162]:
df_candidate = df[(df[keyword] > 0)
                  & (df.ipc_classes.str.count("G06")==0) #Escludo "COMPUTING; CALCULATING; COUNTING"
                  #& (df.ipc_classes.str.count("G01")==0) #Escludo "MEASURING; TESTING"
                  #& (df.ipc_classes.str.count("A61")==0) #MEDICAL OR VETERINARY SCIENCE; HYGIENE
                  #& (df.ipc_classes.str.count("G16")==0)#NFORMATION AND COMMUNICATION TECHNOLOGY [ICT] SPECIALLY ADAPTED FOR SPECIFIC APPLICATION FIELDS
                  #& (df.ipc_classes.str.count("G08")==0)#SIGNALLING
                  #& (df.ipc_classes.str.count("G09")==0)#EDUCATION; CRYPTOGRAPHY; DISPLAY; ADVERTISING; SEALS
                  #& (df.ipc_classes.str.count("H04")==0)#LECTRIC COMMUNICATION TECHNIQUE
                 ].query("title not in @df_selected")[["title", "ipc_classes","abstract", "claims"]]

df_candidate

,title,ipc_classes,abstract,claims
12289,content server for providing application unification for public network applications,H04L29/06,"described is a content server for providing application unification for public network websites and applications. in this application the content server is a security platform that provides protection to an internet publisher from third-party threats, protects from unauthorized data harvesting, ...","what is claimed is:1. a content server for providing application unification for one or more public network sites and applications being accessed by a computing device over a public network, the content server comprising:one or more processing units;computer-readable media storing instructions t..."
14661,methods and apparatus for prevention of surgical site infections,"A61B17/02, A61B17/34, A61K31/7036, A61K9/00, A61M31/00","a surgical access device and methods for facilitating access through an incision or wound to a surgical site in a patient's body comprising an inferior retention member, a superior retention member, and a pliable membrane therebetween. the pliable membrane includes a base layer, a permeable memb...","1. 1.-24. (canceled)25. a surgical method for retracting a tissue, the method comprising: providing a surgical device comprising a superior retention member, an inferior retention member, and a pliable membrane coupled therebetween. inserting the inferior retention member into an wound in a body..."
33867,reagent strips reader for analytes measurement in body fluids connected to a smartphone with emergency function,"G01N33/543, G16H50/50, H04M1/725",a portable monitoring strip reader may include one or more light sensors for reading urine monitoring strips. the monitoring strip reader may collect light intensity information and transmit the information to an application on a portable electronic device for analysis. the application may deter...,"1. a reagent strip reader for analytes connected to a smartphone with emergency function, comprising: a case (2) which has an interior area which houses reagent strips, a cover lid (3) for accessing the inside space and entry opening for strips (4) used for accessing the reagent strips; an elect..."
60442,method and apparatus for estimating monetary impact of cyber attacks,H04L29/06,"a new approach is proposed to support generating and presenting to a user cyber attack monetary impact estimation of a current or future cyber attack, which is used to stop monetary losses or to mitigate monetary impacts. first, both historic data and real time data on monetary impact of current...","1. a system to support cyber attack monetary impact estimation, comprising: a data collection engine running on a host and configured to continuously collect data on monetary impact of current and/or potential cyber attacks from a plurality of data pools over computer networks, computer systems,..."
61966,complete compact,"A45D33/00, A45D33/26, A45D44/00, G01N1/22, G01N1/24",a sensor and display means useful for sensing personal appearance. the invention addresses aroma and presenting the characteristics of the aroma to the user in particular detail. the appearance sensor and display can be integrated with another device associated with determining another aspect of...,"i claim:1. a method for determining aroma comprising: (a) providing at least one means of aroma sensing capable of approximately discerning the presence or absence of at least one characteristic of the aroma, (b) sampling aroma with said aroma sensing means, (c) discerning said aroma characteris..."
72418,use of artificial intelligence to identify novel targets and methodologies for skin care treatment,G16H50/20,use of an artificial intelligence engine to provide skin care treatment is disclosed.,"1. a method of providing skin care treatment, comprising: receiving user-specific information; accessing a data structure containing information reflecting relationships between categories of user-specif

In [131]:
keyword = keywords_cosmetics[7]
keyword

'manicur(e|ing)'

In [133]:
df_candidate = df[(df[keyword] > 0)].query("title not in @df_selected")[["title", "ipc_classes","abstract", "claims"]]

df_candidate

,title,ipc_classes,abstract,claims
20938,automatic nail polish application system and method,"A45D29/00, A45D29/11, A45D29/12, A45D29/14, A45D34/04",the present disclosure relates to a robotic apparatus and methods for automatic nail polish application on natural or artificial finger or toe nails. the robot uses artificial intelligence (al) to identify and paints the nails. the robot uses depth sensors and computer vision to plan the movemen...,"1. a system for automatically polishing a nail of a user including:an end-effector havinga cartridge receiving unit,a cartridge including nail polish, anda nozzle;a sensor, to generate target signals from a target location;a nail determination unit, receiving said target signals, to automaticall..."


In [134]:
df_selected = pd.concat([df_selected, df_candidate])

In [135]:
df_selected

,filename,title,ipc_classes,claims,abstract,filing_date,publication_date,priority_date,assignee,inventors,docdb_family_id,total_cosmetics,cosmetic,beauty,body,skin,nail,fingernail,cleans(e|ing),manicur(e|ing),pedicur(e|ing)varnish,polish,nail(s| |-)*polish,lip(s)?,lip( |-)?stic( |k)+,blush,mascara,make(s| |-)*up,cream,soap,toothpaste,sunscreen,lotion,deodorant,care,lip(s| |-)*(care|polish),skin(s| |-)*(care|polish),nail(s| |-)*(care|polish),baby(s| |-)*(care|polish),anti(-)?ag(e|ing),colour|color,total_other,finger
21464,US2020054018_A1.txt,methods for assessing repellant quality of organic materials and methods and compositions for repelling arthropods,"A01N31/02, A01N31/06, A01N33/10, A01N35/02, A01N35/04, A01N35/06, A01N35/10, A01N37/10, A01N37/18, A01N37/20, A01N37/22, A01N37/36, A01N37/44, A01N37/46, A01N43/12, A01N43/16, A01N43/32, A01N43/38, A01N43/40, A01N43/42, A01N43/50, A01N43/54, A01N43/60, A01N43/78, A01N43/90, A01N47/16, A01N47/42,...","1. a method of repelling an arthropod, comprising exposing the arthropod with a repelling composition, wherein the repelling composition comprises one or more compounds selected from the group consisting of methyl n,n-dimethyl anthranilate, ethyl anthranilate, butyl anthranilate, and 2,3-dimethy...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27784,EP3733278_A1.txt,"moisture control device, moisture control method, program, storage medium, generated substance, product, device and equipment","A23L3/26, A23L3/30, A23L3/32, A23L5/30, A47J37/12, B01J19/08, B01J19/10, B01J19/12","1. a moisture control apparatus, wherein a predetermined voltage or current including a dc component and/or an ac component is applied to at least one electrode that generates at least one of an electric field, a magnetic field, an electromagnetic field, electromagnetic waves, sound waves, and u...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
73457,US2017079274_A1.txt,methods for assessing repellant quality of organic materials and methods and compositions for repelling arthropods,"A01N31/02, A01N31/06, A01N33/10, A01N35/02, A01N35/04, A01N35/06, A01N35/10, A01N37/10, A01N37/18, A01N37/20, A01N37/22, A01N37/36, A01N37/44, A01N37/46, A01N43/12, A01N43/16, A01N43/32, A01N43/38, A01N43/40, A01N43/42, A01N43/50, A01N43/54, A01N43/60, A01N43/78, A01N43/90, A01N47/16, A01N47/42,...","1. 1-21. (canceled)22. a method of repelling an arthropod, comprising exposing the arthropod to a composition comprising one or more compounds selected from the group consisting of:23. the method of claim 22, wherein the composition is applied on the body of one or more vertebrates or one or mor...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
90992,US10292396_B2.txt,methods for assessing repellant quality of organic materials and methods and compositions for repelling arthropods,"A01N31/02, A01N31/06, A01N33/10, A01N35/02, A01N35/04, A01N35/06, A01N35/10, A01N37/10, A01N37/18, A01N37/20, A01N37/22, A01N37/36, A01N37/44, A01N37/46, A01N43/12, A01N43/16, A01N43/32, A01N43/38, A01N43/40, A01N43/42, A01N43/50, A01N43/54, A01N43/60, A01N43/78, A01N43/90, A01N47/16, A01N47/42,...","1. a method of repelling an arthropod, comprising exposing the arthropod to a composition comprising one or more compounds selected from the group consisting of:2. the method of claim 1, wherein the composition is applied on the body of one or more vertebrates or one or more plants to expose the...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
95067,US9491942_B2.txt,methods for assessing repellant quality of organic materials 

In [ ]:
##### VECCHIO CODICE:

In [49]:
df_selected[["filename"]].to_excel("selected_patents_filename_BIG.xlsx", index=False, header=False)

In [50]:
df_selected.to_excel("selected_patents_BIG.xlsx", index=False)

In [417]:
for ipc in very_interesting_ipc_classes:
    try:
        print(f"{ipc}: {ipc_class_dict[ipc].split(' - ')[-1]}")
    except:
        print(f"{ipc}: none")

A45D44/00: A45D44/00: Other cosmetic or personal care articles, e.g. for hairdressers' rooms + 
A45D29/14: A45D29/14: Manicuring or pedicuring implements + motor-driven
A45D29/00: A45D29/00: Manicuring or pedicuring + 
A61K8/00: A61K8/00: Cosmetics or similar toilet preparations + 
A61Q1/00: A61Q1/00: Make-up preparations; Body powders; Preparations for removing make-up + 
A61Q15/00A61Q19/10: none
C11B9/00: C11B9/00: Essential oils; Perfumes + 
A23L27/10: A23L27/10: Spices; Flavouring agents or condiments; Artificial sweetening agents; Table salts; Dietetic salt substitutes; Preparation or treatment thereof + Natural spices, flavouring agents or condiments; Extracts thereof
A45D34/04: A45D34/04: Containers or accessories specially adapted for handling liquid toilet or cosmetic substances, e.g. perfumes + Appliances specially adapted for applying liquid, e.g. using roller or ball
A45D31/00: A45D31/00: Artificial nails + 
A46B9/04: A46B9/04: Arrangements of the bristles in the brush body